In [2]:
import sys
import xlrd
import cv2
import pandas as pd
from PyQt5 import QtWidgets, QtGui, QtCore
from PyQt5.QtWidgets import QMainWindow, QApplication, QWidget, QPushButton, QAction, QLineEdit, QTextEdit, QMessageBox, QLabel, QGridLayout
from PyQt5.QtGui import QIcon, QMovie, QPixmap, QFont, QPainter, QPen
from PyQt5.QtCore import Qt, pyqtSlot, QSize
from PyQt5.QtGui import QMovie

meta = ""
fonts = ""
root = ""

with open('D:\\settings.txt') as f:
    lines = f.readlines()
    for line in lines:        
        if (line.find("meta")!=-1):            
            meta = line[6:].strip()
        if (line.find("root")!=-1):            
            root = line[6:].strip()
        if (line.find("fonts")!=-1):
            #print(line[7:])
            fonts = line[7:].strip()

meta_file = meta
df = pd.read_excel(meta_file, sheet_name = "Master")


In [3]:
def get_FileName(input):   
    s_index = input.find(".png")    
    return input[s_index-5:s_index+4]

def find_end(image):
    w = image.shape[1]
    final = 0
    try:
        for y in range (w-25,0,-1):
            roi = image[0:160,y:y+25]
            sum = 0
            for i in range(160):
                for j in range(25):
                    #print(roi[i,j])
                    k = roi[i,j]
                    if k[0]==255 and k[1]==255 and k[2]==255:
                        flag =0
                    else:
                        flag =1
                    sum = sum+flag
            #rint(y,sum)
            if sum==0:
                final = y
                return final+15
    except:
        a=0
    return final+15



In [4]:
def make_tankrifont(input_text):
    im = cv2.imread(fonts + "F_S.png")
    im_b = cv2.imread(fonts + "F_67.png")


    len_ = len(input_text)
    skip_next = 0

    for index in range (len_):

        im_t = cv2.imread(fonts + "F_67.png")
        im_b = cv2.imread(fonts + "F_67.png")
        prefix = ""
        marker = ""
        if skip_next!=0:
            skip_next=skip_next-1
            continue

        now_ = input_text[index]
        if index+1<len_:
            next_ = ord(input_text[index+1])
            #print("Next is " + str(next_))
            if next_ in (2366,2375,2376,2379,2380,2370,2306):
                #print("Yes")
                marker = "T"
                skip_next = 1
                if (index+2<len_) and (ord(input_text[index+2]) in (2305,2306)):
                    skip_next = 2
                    prefix = "A"
            if next_ in (2369,2370,2381):
                marker = "B"
                skip_next = 1
                if (index+2<len_) and (ord(input_text[index+2]) in (2305,2306)):
                    skip_next = 2
                    marker = "BT"
            if next_ == 2367:
                marker = "LT"
                skip_next = 1
                if (index+2<len_) and (ord(input_text[index+2]) in (2305,2306)):
                    skip_next = 2
                    prefix = "A"
            if next_ == 2368:
                marker = "RT"
                skip_next = 1
                if (index+2<len_) and (ord(input_text[index+2]) in (2305,2306)):
                    skip_next = 2
                    prefix = "A"


        uni_ = ord(now_)       
        x = df.loc[df['Unicode_D'] == uni_]
        filename = (get_FileName(str(x['File_Name']))).strip()
        if (filename.find("png"))==-1:
            print(now_,uni_,filename)
            filename = "F_99.png"

        img = cv2.imread(fonts + filename)

        if (marker != ""):
            xn = df.loc[df['Unicode_D'] == next_]
            filename = (get_FileName(str(xn['File_Name']))).strip()
            #print(now_,uni_,filename)
            #print('D:\\Project Himachal\\Handwritten Sample\\Fonts_Resized\\' + filename)
            if (marker=="T"):
                im_t = cv2.imread(fonts + prefix+filename)
                #print("Hari Om")
            if (marker=="B"):
                im_b = cv2.imread(fonts + filename)

            if (marker=="LT"):
                im_t = cv2.imread(fonts + prefix + 'F_14T.png')
                im_temp = cv2.imread(fonts + 'F_14L.png')
                img = cv2.hconcat([im_temp,img])           

            if (marker=="RT"):
                im_t = cv2.imread(fonts + prefix + 'F_14T.png')
                im_temp = cv2.imread(fonts + 'F_14L.png')
                img = cv2.hconcat([img,im_temp]) 

            if (marker=="BT"):
                im_t = cv2.imread(fonts + 'F_30.png')
                im_b = cv2.imread(fonts + filename)            
        
        if (uni_==2371):
            im_t = cv2.imread(fonts + 'F_14T.png')

        w = img.shape[1]
        top = cv2.resize(im_t, (w, 30))
        bottom = cv2.resize(im_b, (w, 30))
        #print(img.shape)
        if (uni_) not in (2310,2311,2312,2313,2314,2320,2323,2324):
            img = cv2.vconcat([top, img])
        #print(img.shape)
        img = cv2.vconcat([img,bottom])
        #print(img.shape)
        #v2.imshow('temp', img)
        im = cv2.hconcat([im, img])
    return im

def tile_image(image,tile_width):
    
    #tile_width = 5000
    #tile_width = tile_width - 50
    im_b = cv2.imread(fonts + 'F_67.png')
    im_le = cv2.imread(fonts+ 'F_LineEnd.png')
    w = image.shape[1]
    
    img = cv2.resize(im_b, (tile_width, 30))
    
    
    y = 0
    
    while y<w:    
        #print(y)
        if(y+tile_width<w):
            im = image[0:160,y :y + tile_width]
            far_end = find_end(im)
            #print(far_end)
            im = image[0:160,y:y + far_end]
            image_pad = cv2.resize(im_b, (tile_width-far_end, 160))
            #print(im.shape)            
            #print(image_pad.shape)  
            im = cv2.hconcat([im, image_pad])
            pre_ = tile_width-far_end
            #print(im.shape) 
            y = y + far_end
        else:
            im = image[0:160,y:w]
            #print(im.shape)
            #print(w-y)
            image_pad = cv2.resize(im_b, (tile_width-(w-y), 160))
            im = cv2.hconcat([im, image_pad])
            #print(im.shape)
            y=y+tile_width
        img = cv2.vconcat([img,im])
    
    return img

# Use this section only if you need UI to run in the interactive mode

In [ ]:
class AnotherWindow(QWidget):

    def __init__(self,path):
        super().__init__()
        layout = QGridLayout()
        #self.label10 = QLabel("Semantic Class")
        #self.label10.move(50,120)
        
        painter = QPainter(self)
        painter.setBrush(Qt.black)
        painter.setPen(QPen(Qt.black, 2, Qt.SolidLine))
        
        self.label = QLabel(self) 
        self.pixmap = QPixmap(path)
        self.label.setPixmap(self.pixmap)
 
        # Optional, resize label to image size
        self.label.resize(self.pixmap.width(),
                          self.pixmap.height())
                
        # show all the widgets
        self.show()      

class App(QMainWindow):

    def __init__(self):
        super().__init__()
        self.title = 'Takri Project'
        self.left = 50
        self.top = 50
        self.width = 850
        self.height = 200
        self.initUI()
    
    def initUI(self):
        
        self.central_widget = QtWidgets.QWidget(self)
        self.setCentralWidget(self.central_widget)
        self.layout = QtWidgets.QFormLayout(self.central_widget)
        
        self.setWindowTitle(self.title)
        self.setGeometry(self.left, self.top, self.width, self.height)
    
        urlLink="<a href=\"https://translate.google.com\">'Click this link to translate to devanagiri script'</a>" 
        self.label0 = QLabel(urlLink, self)
        #label0.setText(urlLink)
        self.label0.move(600, 10)
        self.label0.setFont(QFont('Arial', 10))
        self.label0.setOpenExternalLinks(True)
        
        self.label = QLabel('Please enter only Devanagigi(देवनागिरी) text into the text box. Click on above URL if you need help in translation to Devanagiri', self)
        self.label.move(40, 30)
        self.label.setFont(QFont('Arial', 12))
        
        self.textbox = QLineEdit(self)        
        self.textbox.move(40, 65)
        self.textbox.resize(780,80)
        font = self.textbox.font()      # lineedit current font
        font.setPointSize(32)               # change it's size
        self.textbox.setFont(font)      # set font
        
        self.button_main = QPushButton('Build Takri Lipi', self)
        self.button_main.move(250,170)
        self.button_main.resize(300,30)
        
        self.button_main.clicked.connect(self.on_click_main_button)
        
        self.layout.addRow(self.label0)
        self.layout.addRow(self.label)
        self.layout.addRow(self.textbox)
        self.layout.addRow(self.button_main)
        
        
        
        self.show()
        
        
    
    def closeEvent(self, e):
        close = QMessageBox.question(self, "Close Application","Are you sure?",QMessageBox.Yes | QMessageBox.No,QMessageBox.No)
        if close == QMessageBox.Yes:
            sys.exit (app.exec_ ())

   

    
    def on_click_main_button(self):       
        
        textboxValue = self.textbox.text()
        text = ""
        for t in textboxValue:
            if not (ord(t)==2364):
                text = text+t             

        takri_image = make_tankrifont(text)
        im2 = tile_image(takri_image,2000)        
        
        
        half = cv2.resize(im2, (0, 0), fx = 0.5, fy = 0.5)
        cv2.imwrite(root + 'Test.png',half)
        path = root + "Test.png"
        self.window = AnotherWindow(path)
       
        
        
        #msg = QMessageBox.question(self, "The input text has been converted \n to Takri and saved on disc.", "Text Converted. Want to convert more?",QMessageBox.Yes | QMessageBox.No,QMessageBox.No)
        
        #if msg == QMessageBox.No:
            #sys.exit (app.exec_ ())
        
if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = App()
    sys.exit(app.exec_())

In [9]:
line_number = 0
page_number = 0
title = 0
blankLine = cv2.imread("D:\\Project Himachal\\Data\\Kangri\\BlankLine.png")
im = blankLine

with open('D:\\Project Himachal\\Data\\Kangri\\KangriPoems.txt', encoding='utf-8') as f:
    lines = f.readlines()    
    for line in lines: 
        line = "  " + line
        if (line_number==0):
            im = blankLine                        
            im = cv2.vconcat([im,blankLine])             
            line_number = line_number + 1
            
        if (line_number==20) or (line.strip()=="New"):            
            im = cv2.vconcat([im,blankLine])  
            for blanks in range(20-line_number):
                im = cv2.vconcat([im,blankLine]) 
            if(page_number>0):
                im = cv2.vconcat([im,tile_image(make_tankrifont(str(page_number)),3000)]) 
            #im = cv2.resize(im, (600, 1000))
            cv2.imwrite('D:\\Project Himachal\\Data\\Kangri\\Page_' + str(page_number) +'.png', im  )
            if (line.strip()=="New"):
                title=1
                im = blankLine
                im = cv2.vconcat([im,blankLine]) 
                im = cv2.vconcat([im,blankLine]) 
                im = cv2.vconcat([im,blankLine]) 
                im = cv2.vconcat([im,blankLine]) 
                im = cv2.vconcat([im,blankLine]) 
                line_number = 5
            else:
                im = blankLine
                im = cv2.vconcat([im,blankLine]) 
                line_number = 1
            page_number = page_number +1
            
        line_number = line_number + 1
        #print("Processing " + line)
        try:                    
            im = cv2.vconcat([im,tile_image(make_tankrifont(line),3000)])        
        except:
            #print("In exception")
            im = tile_image(make_tankrifont(line),3000)        
        
        if (title >0):
            im = cv2.vconcat([im,blankLine])
            line_number = line_number + 1
            title=0
        


 10 

 10 

 10 
" 34 
" 34 

 10 
" 34 
" 34 

 10 
" 34 
" 34 

 10 
" 34 
" 34 

 10 
" 34 
" 34 

 10 
" 34 
" 34 

 10 
" 34 
" 34 

 10 
" 34 
" 34 

 10 
" 34 
" 34 

 10 
" 34 
" 34 

 10 
" 34 
" 34 

 10 
" 34 
" 34 

 10 
" 34 
" 34 

 10 
" 34 
" 34 

 10 
" 34 
" 34 

 10 

 10 
N 78 
e 101 
w 119 

 10 
" 34 
" 34 

 10 

 10 

 10 
- 45 

 10 
- 45 

 10 

 10 

 10 

 10 

 10 

 10 

 10 
- 45 

 10 
़ 2364 

 10 

 10 

 10 

 10 
- 45 

 10 
़ 2364 
- 45 

 10 

 10 
़ 2364 
- 45 

 10 
- 45 

 10 

 10 
़ 2364 

 10 
़ 2364 

 10 

 10 

 10 

 10 
- 45 

 10 
- 45 

 10 
़ 2364 

 10 

 10 

 10 

 10 

 10 

 10 
़ 2364 
़ 2364 
़ 2364 

 10 
़ 2364 

 10 

 10 

 10 
- 45 

 10 
़ 2364 

 10 

 10 

 10 
़ 2364 

 10 
़ 2364 

 10 

 10 

 10 
़ 2364 

 10 
? 63 

 10 
़ 2364 

 10 
़ 2364 

 10 
़ 2364 

 10 
़ 2364 

 10 

 10 

 10 
़ 2364 

 10 

 10 
़ 2364 

 10 
- 45 

 10 
़ 2364 
- 45 
़ 2364 

 10 

 10 
- 45 

 10 

 10 

 10 
़ 2364 

 10 

 10 

 10